In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from torch.nn import functional as F
import random

model_name = 'flax-community/papuGaPT2'
device = 'cuda'

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

def log_probs_from_logits(logits, labels):
    logp = F.log_softmax(logits, dim=-1)
    logp_label = torch.gather(logp, 2, labels.unsqueeze(2)).squeeze(-1)
    return logp_label

def sentence_prob(sentence_txt):
    input_ids = tokenizer(sentence_txt, return_tensors='pt')['input_ids'].to(device)
    with torch.no_grad():
        output = model(input_ids=input_ids)
        log_probs = log_probs_from_logits(output.logits[:, :-1, :], input_ids[:, 1:])
        seq_log_probs = torch.sum(log_probs)
    return seq_log_probs.cpu().numpy()  

In [2]:
from itertools import permutations
words = "babuleńka miała dwa rogate koziołki".split()
sorted(permutations(words), key=lambda l: sentence_prob(" ".join(l)), reverse=True)[:10]

[('miała', 'babuleńka', 'dwa', 'rogate', 'koziołki'),
 ('miała', 'babuleńka', 'dwa', 'koziołki', 'rogate'),
 ('dwa', 'rogate', 'koziołki', 'miała', 'babuleńka'),
 ('dwa', 'babuleńka', 'miała', 'koziołki', 'rogate'),
 ('miała', 'dwa', 'koziołki', 'rogate', 'babuleńka'),
 ('miała', 'koziołki', 'babuleńka', 'dwa', 'rogate'),
 ('miała', 'dwa', 'rogate', 'koziołki', 'babuleńka'),
 ('miała', 'babuleńka', 'rogate', 'dwa', 'koziołki'),
 ('miała', 'dwa', 'koziołki', 'babuleńka', 'rogate'),
 ('dwa', 'rogate', 'koziołki', 'babuleńka', 'miała')]